### Unit 1 Homework:  Scraping the Yelp Website

Welcome!  For this homework assignment you'll be tasked with building a web scraper in a manner that builds on what was covered in our web scraping class.

The assignment will extend the lab work done during that time, where we built a dataset that listed the name, number of reviews and price range for restaurant on the following web page: https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C%20United%20Kingdom&ns=1

**What You'll Turn In:**

A finished jupyter notebook that walks us through the steps you took in order to get your results.  Provide notes where appropriate to explain what you are doing.

The notebook should produce a finished dataset at the end.  

If for some reason you're experiencing problems with the final result, please let someone know when turning it in.
 
The homework is divided into five tiers, each of which have increasing levels of difficulty:

##### Tier 1: Five Columns From the First Page

At the most basic level for this assignment, you will need to extend what we did in class, and create a dataset that has five columns in it that are 30 rows long.  This means you will not need to go off the first page in order to complete this section.

##### Tier 2:  100 Row Dataset With At Least 3 Columns

For this portion of the assignment, take 3 of your columns from step 1, and extend them out to multiple pages on the yelp website.  You should appropriately account for the presence of missing values.

##### Tier 3:  100 Row Dataset With At Least 5 Columns

Very similar to Tier 2, but if you use this many columns you will be forced to encounter some columns that will frequently have missing values, whereas with Tier 2 you could likely skip these if you wanted to.  

##### Tier 4:  100 Row Dataset With At Least 5 Columns + Individual Restaurant Categories

Restaurants often have different categories associated with them, so grabbing them individually as separate values is often challenging.  To complete this tier, you'll have to find a way to 'pick out' each of the individual categories as their own separate column value.  

##### Tier 5:  Unlimited Row Dataset With At Least 5 Columns + Individual Restaurant Categories

Take what you did in Tier 4, and extend it so that the code will work with an arbitrary number of pages.  Ie, regardless of how many pages there are listing the best restaurants in London, your scraper will find them, and cleanly parse their information into clean datasets.

### Hints

Here are a few tips that will save you time when completing this assignment:

 - The name, average rating, total ratings and neighborhood of a restaurant tend to be the 'easy' ones, because they rarely have missing values, so what ever logic you use on the first page will typically apply to all pages.  They are a good place to start
 - Phone numbers, price ranges and reviews are more commonly missing, so if you are trying to get a larger number of items from them across multiple pages you should expect to do some error handling
 - You can specify any sort of selector when using the `find_all()` method, not just `class`.  For example, imagine you have the following `<div>` tag:
    `<div class='main-container red-blue-green' role='front-unit' aria-select='left-below'>Some content here</div>`
    
   This means that when you use `scraper.find_all('div')`, you can pass in arguments like `scraper.find_all('div', {'role': 'front-unit'})` or anything else that allows you to isolate that particular tag.
 - When specifying selectors like `{'class': 'dkght__384Ko'}`, sometimes less is more.  If you include multiple selectors, you are saying return a tag with **any one of these** distinctions, not all of them.  So if your results are large, try different combinations of selectors to get the smallest results possible.
 - If you begin dealing with values that are unreliably entered, you should use the 'outside in' technique where you grab a parent container that holds the element and find a way to check to see if a particular value is there by scraping it further.  The best way to do this is to try and find a unique container for every single restaurant.  This means that you will have a reliable parent element for every single restaurant, and within *each of these* you can search for `<p>`, `<a>`, `<div>`, and `<span>` tags and apply further logic.
 - When you get results from `BeautifulSoup`, you will be given data that's denoted as either `bs4.element.Tag` or `bs4.element.ResultSet`.  They are **not the same**.  Critically, you can search a `bs4.element.Tag` for further items, but you cannot do this with a `bs4.element.ResultSet`.  
 
   For example, let's say you grab all of the divs from a page with `scraper.find_all('div')` and save it as the variable `total_divs`.  This means `total_divs` will look somethig like this:  
   
   `[<div><p>Div content</p><p>Second paragraph</p></div>,`
      `<div><p>Div content</p><p>Second paragraph</p></div>,`
      `<div><p>Div content</p><p>Second paragraph</p></div>]`
      
   In this case the variable `total_divs` is a result set and there's nothing else you can do to it directly.  However, every item within `total_divs` is a tag, which means you can scrape it further.  
   
   So if you wanted you could write a line like:  `total_paragraphs = [div.find_all('p') for div in total_divs]`, and get the collection of paragraphs within each div.  
   
   If you confuse the two you'll get the following error message:  
   
   `AttributeError: ResultSet object has no attribute 'find_all'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?`
 - The values of the different selectors change periodically on yelp, so if your scraper all of a sudden stops working that's probably why.  Ie, if you have a command like `scraper.find_all('div', {'class': '485dk0W__container09'}` that no longer returns results, the class `485dk0W__container09` may now be `r56kW__container14` or something similar.

In [2]:
# your code here
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [3]:
url = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C%20United%20Kingdom&ns=1'

In [7]:
res = requests.get(url)

In [8]:
soup = BeautifulSoup(res.content, 'html.parser')

In [24]:
pages = int(soup.find('div', attrs={'class':'pagination__09f24__q1J0W'}).find(
    'span', attrs={'class':'css-e81eai'}).text.split()[-1])
pages

24

In [128]:
def has_rating(tag):
    if tag.has_attr('aria-label'):
        if 'star rating' in tag['aria-label']:
            return True
    return False

def get_restaurant_data(restaurant):
    data = {
    'name':None,
    'rating':None,
    'reviews':None,
    'price':None,
    'categories':None,
    'neighborhood':None
    }
    
    #name
    item = restaurant.find('a',attrs={'class':'css-166la90'})
    if item:
        data['name'] = item['name']

    #rating
    item = restaurant.find(has_rating)
    if item:
        data['rating'] = float(item['aria-label'][:-12])

    #reviews
    item =  restaurant.find('span', attrs={'class':'reviewCount__09f24__EUXPN'})
    if item:
        data['reviews'] = int(item.text)

    #price range - number of Brittish pound signs
    item = restaurant.find('span', attrs={'class':"priceRange__09f24__2O6le"})
    if item:
        data['price'] = len(item.text)

    #category
    item =  restaurant.find('div', attrs={'class':'priceCategory__09f24__2IbAM'})
    if item:
        cats = item.find_all('a')
        data['categories'] = ','.join(cat.text for cat in cats)

    #neighborhood
    item = restaurant.find('div',attrs={'class':'secondaryAttributes__09f24__3db5x arrange-unit__09f24__1gZC1 border-color--default__09f24__R1nRO'})
    if item:
        data['neighborhood'] = item.find_all('p')[-1].text
        
    return data

In [131]:
# page = 4
# url = f'https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C%20United%20Kingdom&ns=1&start={10*page}'
# res = requests.get(url)
# soup = BeautifulSoup(res.content, 'html.parser')
# containers = soup.find_all('div', attrs={'class':"container__09f24__21w3G"})
# len(containers)

10

In [132]:
restaurant_data = []
for page in range(pages):
    url = f'https://www.yelp.com/search?find_desc=Restaurants&find_loc=London%2C%20United%20Kingdom&ns=1&start={page*10}'
    res = requests.get(url)
    soup = BeautifulSoup(res.content, 'html.parser')
    containers = soup.find_all('div', attrs={'class':"container__09f24__21w3G"})
#     print(page, len(containers), len(restaurant_data))

    for restaurant in containers:
        restaurant_data.append(get_restaurant_data(restaurant))
    
yelp = pd.DataFrame(restaurant_data)
yelp.head()

0 10 0
1 10 10
2 10 20
3 10 30
4 10 40
5 10 50
6 10 60
7 10 70
8 10 80
9 10 90
10 10 100
11 10 110
12 10 120
13 10 130
14 10 140
15 10 150
16 10 160
17 10 170
18 10 180
19 10 190
20 10 200
21 10 210
22 10 220
23 10 230


,name,rating,reviews,price,categories,neighborhood
0,The Breakfast Club,4.0,494,2.0,"Coffee & Tea,Breakfast & Brunch,American (Trad...",Soho
1,The Mayfair Chippy,4.5,282,2.0,Fish & Chips,Mayfair
2,Dishoom,4.5,547,2.0,Indian,Soho
3,Dishoom,4.5,1840,2.0,Indian,Covent Garden
4,Flat Iron,4.5,380,2.0,Steakhouses,Soho


In [133]:
yelp.shape

(240, 6)

In [134]:
yelp.isna().sum()

name             0
rating           0
reviews          0
price           39
categories       0
neighborhood     0
dtype: int64

In [145]:
yelp = pd.concat([yelp,
    yelp.categories.str.split(',', expand=True).rename(columns={0:'category_1', 1:'category_2', 2:'category_3'})],
    axis=1).drop(
    columns='categories'
    )

yelp

,name,rating,reviews,price,neighborhood,category_1,category_2,category_3
0,The Breakfast Club,4.0,494,2.0,Soho,Coffee & Tea,Breakfast & Brunch,American (Traditional)
1,The Mayfair Chippy,4.5,282,2.0,Mayfair,Fish & Chips,None,None
2,Dishoom,4.5,547,2.0,Soho,Indian,None,None
3,Dishoom,4.5,1840,2.0,Covent Garden,Indian,None,None
4,Flat Iron,4.5,380,2.0,Soho,Steakhouses,None,None
...,...,...,...,...,...,...,...,...
235,Albertini Restaurant,4.5,52,1.0,Euston,Pizza,Italian,Seafood
236,Eggslut,4.0,6,NaN,Notting Hill,Burgers,Sandwiches,None
237,London House by Gordon Ramsay,4.5,23,3.0,Battersea,Modern European,None,None
238,Chicken Shop,5.0,24,2.0,Covent Garden,Chicken Shop,None,None
